Размышления в вольном стиле.
При первом взгляде на задачу в памяти всплывает словосочетание "временные ряды".
В самом деле - имеем данные ("наблюдения") полученные через равные промежутки времени. Но наблюдаемых объектов много - т.е. в каждый момент времени имеем несколько ("много") значений.
Т.е. уже не временной ряд а "panel data". Которые панельные данне анализировать посложнее чем временной ряд. С другой стороны проще всё таки рассмотреть данные по каждому из пользователей как независимый
от других времменной ряд. Но независимый ли? Но предположим, что независимый - в самом деле как зависит оплата за услуги одного пользователя от оплаты другим, и будем что то делать с данными по каждому пользователю (которые данные - попросту некоторая последовательность чисел) независимо от данных по всем другим. И тут самая главная проблема - моментов времени в которые производились "наблюдения" немного, <= 12. Т.е. о каких то "продвинутых" техниках работы с временными рядами, выявления периодической, сезонной частей говорить не приходится. С трудом можно (наверное) использовать скользящее среднее. Напрашивается вывод что "временнорядные" техники работы с этими данными нам практически без надобности.
Тогда как будем строить prediction?  Простейший вариант - считать оплату линейной функцией времени. Ну и использовать линейную регрессию для нахождения оной. По каждому пользователю. А если оплата им, пользователем, услуг от месяца к месяцу сильно колеблеться? Использовать нелинейную функцию времени в качестве аппроксимирующей? Интуиция подсказывает, что в данном случае это не есть хорошее решение. Ну хотя бы, вот такое рассуждение "на глазок": для "хорошего" пользователя - у которого оплата не "скачет" сильно - и линейная регрессия сойдёт, а для "нехорошего" пользователя придется подбирать другую причём "индивидуальную" аппроксимирующую функцию (напр. полином степени отличной от 1-й). Ключевое слово здесь "индивидуальная" - т.е. для каждого пользователя надо подобрать аппроксимирующую функцию. Для каждого! Нет, можно, конечно поступить проще - воспользоваться какой нибудь интерполяционной формулой (напр. Лагранжа) и найти многочлен который точно "попадёт" во все точки на плоскости "время - оплата" с независимой переменной "время". Но что то подсказывает что это решение совсем никуда не годится (Да и вообще это не задача интерполяции - время у нас дискретно, промежуточных значений между точками времени "март" и "апрель" нет, ). Опять же, сумму оплаты по всем пользователям на следующий месяц скорее всего можно довольно точно предсказать воспользовавшись линейной регрессией (А были бы данные за "большой" промежуток времени или точнее "много" (> 100)  моментов времени в которые производились наблюдения, то можно было бы и time series forecasting применить). Но это несколько не то что нам нужно. Нужно по каждому пользователю дать prediction.
Ну, вообщем, и так уже много написанно, так что лучше сразу перейти к идее решения. Будем рассматривать какую-то часть (подпоследовательность то есть несколько последовательных значений) набора данных по каждому "пользователю" в качестве независимых переменных а значение (неизвестное) которое необходимо "узнать" - в качестве зависимой переменной. В предельном варианте вся известные N значений по каждому пользователю являються конкретными значениями N независимых переменных а неизвестное значение величины оплаты в следующий месяц  - зависимой переменной.  И соответственно, есть некая функция которая на основе данных значений независимых переменных вычисляет зависимую - "следующее" значение оплаты. Тут сразу стоит заметить что в данном случае мы, так сказать, опускаем за ненадобностью "временнУю" природу данных.
Осталось дело за малым - найти эту функцию от n <= N переменных.Ну а это уже отчётливо поставленная задача машинного обучения. Надо построить нейронную сеть (т.к. при помощи таковой можно аппроксимировать практически любую функцию) принимающую на вход "предыдущие" значения оплаты и выдающую на выходе "последующее" значение. Для тренировки такой сети можно брать подпоследовательность значений оплаты, например за январь, февраль, ..., июль и вычислять значение за август (которое у нас уже имеется. Положим что у нас есть значения с января по август). Далее "скормить" натренированной сети значения за февраль - август и получить значение за сентябрь которое нам и нужно предсказать. С длинной тренировочной подпоследовательности можно поэкспериментировать. Может быть, стоило бы поэкспериментировать с реккурентными нейронными сетями.
И последнее. Само собой нам не важно с какими данными в данном случае надо иметь дело - с помесячной оплатой пользователя за объём отправленной почты, или с годовым удоем молока по фермам.

In [81]:
import sys
import warnings
import pickle

import numpy as np
import pandas as pd
from pandas import Series, DataFrame

from numpy.testing import assert_almost_equal, assert_array_equal

from sklearn import cross_validation
from sknn.mlp import Regressor, Layer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.grid_search import  GridSearchCV
from sklearn.utils.testing import (assert_raises, assert_greater,
                                   assert_equal, assert_false)

In [82]:
np.seterr(all='warn')
scaler = StandardScaler()

In [83]:
payment_data = pd.read_csv('user_payments.csv', index_col=0)
payment_data.head(4)

,January,February,March,April,May,June
User,,,,,,
1,73.65,91.26,118.50,111.45,110.42,131.20
2,118.15,127.74,146.49,163.43,163.60,169.54
3,102.44,129.94,134.28,132.59,140.58,182.04
4,405.04,489.13,526.05,545.83,570.26,543.84


Как мы видим, данных у нас всего за шесть месяцев что гораздо меньше 12

In [84]:
payment_data_matrix = payment_data.as_matrix()
payment_data_matrix = payment_data_matrix.astype(np.float64)

Для удобства всё связанное с обучением нашего multi layer perceptron-а выносим в отдельную функцию.

In [85]:
def train_regression_predictor(train_x, train_y, learning_rule='sgd', learning_rate=0.002, n_iter=20, units=4):
    mlp = Regressor(layers=[Layer('Rectifier', units=units),
                            Layer('Linear')],
                   learning_rule=learning_rule,
                   learning_rate=learning_rate,
                   n_iter=n_iter)
    mlp.fit(train_x, train_y)
    print mlp.score(train_x, train_y)
    return mlp

In [86]:
#Выделяем наши независимые переменны.
#Это данные по каждому из пользователей с января по апрель
#(включительно), всего четыре переменные.
independed_vars_values = payment_data_matrix[0:,:4]

In [87]:
#Выделяем независимую переменную - данные за май
depended_var_values = payment_data_matrix[0:, 4]

In [88]:
train_x, test_x, train_y, test_y = cross_validation.train_test_split(independed_vars_values, depended_var_values, test_size=0.3)

In [89]:
train_x = scaler.fit_transform(train_x)
test_x = scaler.fit_transform(test_x)

In [90]:
payment_mlp = train_regression_predictor(train_x, train_y, learning_rule='sgd', learning_rate=0.0002, n_iter=20, units=4)

0.964160850092


In [91]:
#pickle.dump(payment_mlp, open('payment_mlp.pkl', 'wb'))
score = payment_mlp.score(test_x, test_y)

In [92]:
# Выделяем "реальные" (февраль-май) значения независмых переменных
# на которых обученная нейронная сеть будет вычислять оплату
# на следующий месяц (июнь) 
real_x = payment_data_matrix[0:, 1:5]
real_x = scaler.fit_transform(real_x)

In [93]:
# А вот пришли "реальные" данные за июнь
real_y = payment_data_matrix[0:, 5]

In [94]:
# Немного покроссвалидируем
best_mlp = payment_mlp
for i, (train_indices, test_indices) in enumerate(cross_validation.KFold(depended_var_values.size, n_folds=3, shuffle=True)):
    train_x = independed_vars_values[train_indices]
    test_x = independed_vars_values[test_indices]
    train_y = depended_var_values[train_indices]
    test_y = depended_var_values[test_indices]
    train_x = scaler.fit_transform(train_x)
    test_x = scaler.fit_transform(test_x)
    payment_mlp = train_regression_predictor(train_x, train_y, learning_rule='sgd', learning_rate=0.0002, n_iter=20, units=10)
    pickle.dump(payment_mlp, open('payment_mlp_cv_0%s_sgd.pkl' % str(i), 'wb'))
    crossvalidation_score = payment_mlp.score(test_x, test_y)
    if crossvalidation_score > score:
        score = crossvalidation_score
        best_mlp = payment_mlp
    
print 'predicted values and real values score', best_mlp.score(real_x, real_y)

0.894701052346
0.966478555986
0.83712155004
predicted values and real values score 0.943858998779


In [95]:
predicted_y = best_mlp.predict(real_x)
predicted_y = predicted_y.reshape(predicted_y.shape[0],)

In [96]:
predicted_and_true_values = DataFrame({'predicted' : predicted_y.round(decimals=2), 'real' : real_y})
predicted_and_true_values.head(20)

,predicted,real
0,118.66,131.20
1,180.99,169.54
2,151.78,182.04
3,674.23,543.84
4,157.22,125.72
5,186.58,169.95
6,266.15,236.43
7,222.94,216.86
8,696.27,608.13
9,334.88,283.80


In [97]:
predicted_and_true_values.to_csv('predicted_and_true_values.csv')

In [112]:
# Поэкспериментируем с автоматическим подбором параметров обучаемого 
# персептрона используя встроенные средства scikit-learn 

units = range(4, 15)
learning_rate = [0.00002, 0.00001, 0.000005]
layer_type = ['Rectifier', 'Sigmoid', 'Tanh', 'ExpLin']
param_grid = {'learning_rate' : learning_rate,
              'nonlinear__units' : units,
              'nonlinear__type' : layer_type,
             }
nonlinear_layer = Layer('Rectifier', units=4, name='nonlinear')
tuned_regressor_mlp = Regressor(layers=[nonlinear_layer,
                                        Layer('Linear')],
                                learning_rate=0.00002,
                                n_iter=10)
x = scaler.fit_transform(independed_vars_values)
y = depended_var_values
gs = GridSearchCV(tuned_regressor_mlp, param_grid=param_grid, cv=3)
gs.fit(x, y)
print gs.best_params_, gs.best_score_
best_mlp_by_gs = gs.best_estimator_

{'nonlinear__units': 7, 'learning_rate': 2e-05, 'nonlinear__type': 'ExpLin'} 0.925236379552


In [113]:
print 'predicted by best gs mlp values and real values score', best_mlp_by_gs.score(real_x, real_y)

predicted by best gs mlp values and real values score 0.914137876313


In [114]:
predicted_y = best_mlp_by_gs.predict(real_x)
predicted_y = predicted_y.reshape(predicted_y.shape[0],)
predicted_and_true_values = DataFrame({'predicted' : predicted_y.round(decimals=2), 'real' : real_y})
predicted_and_true_values.to_csv('predicted_by_best_gs_mlp_and_true_values.csv')
predicted_and_true_values.head(20)

,predicted,real
0,116.96,131.20
1,165.45,169.54
2,141.54,182.04
3,585.43,543.84
4,147.27,125.72
5,170.95,169.95
6,234.27,236.43
7,201.11,216.86
8,606.28,608.13
9,299.55,283.80


На самом деле данные по "пользователям" и их "ежемесячной оплате" - это несколько видоизменённый  датасет "Spanish Dairy Farm Production" взятый вот здесь http://people.stern.nyu.edu/wgreene/Econometrics/PanelDataSets.htm. Так что наши "пользователи" - это на самом деле фермы, "месяцы" - года с 93 по 98 а "оплата" - количество произведённого молока (делённое на 1000). Хотелось всё таки поработать с более-менее реальными данными подходящими под условия задачи.
Желательно бы конечно работать с данными о большем количестве наблюдаемых объектов ("пользовтелей") - несколько тысяч, данный датасет несколько маловат, и вряд ли на нём можно обучить наш персептрон достато хорошо. Ну и конечно же хотелось бы большего количества наблюдений, что бы, поэкспериментировать, например, с количеством независимых переменных да и вообще в данном случае больше - лучше.
Сгенерировать же условный датасет, по каким то критериям близкий к реальному - само по себе непростая задача. Хороший генератор реальных датасетов - реальный мир. 